In [1]:
import pandas as pd

In [2]:
bandwidth = pd.read_excel('ToyData.xlsx',sheet_name='Inter-Datacenter Links')
dcd = pd.read_excel('ToyData.xlsx',sheet_name='Data Center Details')
slot = dcd.loc[:,['DC','Num of Slots']].dropna(how='any')
location = dcd.loc[:,['Data Partition','Location']]

In [3]:
RQ = pd.read_excel('ToyData.xlsx',sheet_name='Job List').iloc[:,1:]
RQ.set_index(list(RQ)[0],inplace=True)
pre = RQ[list(RQ)[-1]].dropna()
exe = RQ[list(RQ)[-2]]
RQ = RQ.iloc[:,0:-2]

In [4]:
pre = pre.apply(lambda x: x.replace("{","").replace("}","")).str.split(", ",expand=True).transpose()

In [5]:
import re

In [6]:
dag = {}
for c in list(pre):
    for l in pre.index.to_list():
        cell = pre[c][l]
        if(cell and cell!=" "):
            edge = re.search(r'\((?P<start>.+),(?P<end>.+)\)',cell,re.S).groupdict()
            if(edge['start'] in dag):
                dag[edge['start']].append(edge['end'])
            else:
                dag[edge['start']] = [edge['end']]

In [7]:
# topological sort
topo_order = []

def dfs(key):
    if(key in topo_order):
        return
    topo_order.append(key)
    if(key in dag):
        for child in dag[key]:
            dfs(child)

for key in dag:
    dfs(key)

# BFS through topological order
stage = {}
visitqueue = list()
for node in topo_order:
    visitqueue.append(node)
    while(len(visitqueue)>0):
        top = visitqueue.pop(0)
        if(top not in stage):
            stage[top] = 0
        if(top in dag):
            for child in dag[top]:
                visitqueue.append(child)
                if((child not in stage) or \
                    (stage[top] + 1 > stage[child])):
                    stage[child] = stage[top] + 1

stage

{'tB1': 0,
 'tB2': 1,
 'tC1': 0,
 'tC2': 1,
 'tC3': 2,
 'tD1': 0,
 'tD3': 1,
 'tD5': 2,
 'tD2': 0,
 'tD4': 1,
 'tE1': 0,
 'tE2': 1,
 'tE3': 1,
 'tE4': 2,
 'tE5': 2,
 'tE6': 3,
 'tF1': 0,
 'tF2': 1,
 'tF3': 1,
 'tF4': 1,
 'tF5': 2,
 'tF6': 2,
 'tF7': 3,
 'tF8': 3,
 'tF9': 4}

In [8]:
import gurobipy as gp
from gurobipy import GRB